In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/sedroc/Bachelor/FEPCEU')
from enhanced_experiment import EnhancedTimeSeriesExperiment
from models.arima import Arima
from models.baseline import Baseline
from models.chronos import Chronos
from models.lgbm import LGBM
from models.neuralproph import Neuralprophet

from data.data_retriever import Dataretreiver

/home/sedroc/Bachelor/FEPCEU/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [3]:
data = Dataretreiver()
df = data.combined

In [19]:
# Define models
models = {
    #'BASELINE': Baseline(),
    #'LGBM': LGBM(),
    #'ARIMA': Arima(order=[1,1,1], seasonal_order=[1,1,1,24]),
    #'CHRONOS': Chronos(),
    'NEURALPROPHET': Neuralprophet()
}

In [20]:
# Initialize the experiment
experiment = EnhancedTimeSeriesExperiment(
    models=models,
    target_column='price',
    forecast_horizon=168,
    n_splits=1
)


# Define custom combinations of different measurement types with different areas
custom_combinations = [
    # Combination 1: temp area 1, wind area 1, sun areas 1,2,3
    #{
    #    'temp': {1},
    #    'wind': {1},
    #    'sun': {1, 2, 3}
    #},
    # Combination 2: temp areas 1,2, wind area 1, sun area 1
    #{
    #    'temp': {1, 2},
    #    'wind': {1},
    #    'sun': {1}
    #},
    # Add more combinations as needed...
]

# Run the experiments with these custom combinations
experiment.run_feature_group_experiments(
    df=df,
    custom_feature_combinations=custom_combinations
)

Identified area columns: {'sun': {1: 'sun_area_1', 10: 'sun_area_10', 11: 'sun_area_11', 12: 'sun_area_12', 13: 'sun_area_13', 14: 'sun_area_14', 15: 'sun_area_15', 16: 'sun_area_16', 17: 'sun_area_17', 18: 'sun_area_18', 19: 'sun_area_19', 2: 'sun_area_2', 20: 'sun_area_20', 21: 'sun_area_21', 22: 'sun_area_22', 23: 'sun_area_23', 24: 'sun_area_24', 25: 'sun_area_25', 26: 'sun_area_26', 27: 'sun_area_27', 28: 'sun_area_28', 29: 'sun_area_29', 3: 'sun_area_3', 30: 'sun_area_30', 31: 'sun_area_31', 32: 'sun_area_32', 33: 'sun_area_33', 34: 'sun_area_34', 35: 'sun_area_35', 36: 'sun_area_36', 37: 'sun_area_37', 38: 'sun_area_38', 39: 'sun_area_39', 4: 'sun_area_4', 40: 'sun_area_40', 5: 'sun_area_5', 6: 'sun_area_6', 7: 'sun_area_7', 8: 'sun_area_8', 9: 'sun_area_9'}, 'wind': {1: 'wind_area_1', 10: 'wind_area_10', 11: 'wind_area_11', 12: 'wind_area_12', 13: 'wind_area_13', 14: 'wind_area_14', 15: 'wind_area_15', 16: 'wind_area_16', 17: 'wind_area_17', 18: 'wind_area_18', 19: 'wind_area_1

Finding best initial lr: 100%|██████████| 241/241 [00:02<00:00, 97.62it/s] 
WARNING - (py.warnings._showwarnmsg) - /home/sedroc/Bachelor/FEPCEU/.venv/lib/python3.13/site-packages/lightning_fabric/utilities/cloud_io.py:51: UserWarning:

Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.




Predicting DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 117.57it/s]
                      ds           y       yhat1  yhat1 5.0%  yhat1 95.0%  \
0    2024-01-01 00:00:00  126.660004  560.517822  172.415253   781.198242   
1    2024-01-01 01:00:00  209.779999  545.697876  151.114166   775.637451   
2    2024-01-01 02:00:00  198.740005  539.537231  149.547211   763.993042   
3    2024-01-01 03:00:00   30.860001  534.259033  147.134064   750.516724   
4    2024-01-01 04:00:00   -0.220000  540.174072  154.647614   755.693604   
...                  ...         ...         ...         ...          ...   
4503 2024-07-06 15:00:00         NaN  287.665009   46.838470   592.077515   
4504 2024-07-06 16:00:00         NaN  344.381866   88.869598   650.681152   
4505 2024-07-06 17:00:00         NaN  451.164642  193.554718   723.160156   
4506 2024-07-06 18:00:00         NaN  569.839844  298.237274   851.004395   
4507 2024-07-06 19:00:00         NaN  652.519409  345.638214  1010.286499   

   

In [6]:
# Get a table of results
results_df = experiment.summarize_feature_group_results(metric='MAE')
print(results_df)

# Plot the results
experiment.plot_feature_group_results(metric='MAE')

          NEURALPROPHET
base_only           N/A


In [7]:
# Plot predictions for the 'sun_areas_1_2_3' feature group
experiment.plot_feature_group_predictions(
    df=df,
    feature_group='custom_temp:[1]_wind:[1]_sun:[1,2,3]',
    fold_index=0  # First CV fold
)

No predictions available for model NEURALPROPHET in feature group custom_temp:[1]_wind:[1]_sun:[1,2,3], fold 0
